## Implementation of MD5 hashing algorithm
### Based on wiki article and original paper
Author: Mikołaj Koszowski 274392\
Date: 02.02.2020

* https://en.wikipedia.org/wiki/MD5
* https://www.ietf.org/rfc/rfc1321.txt

Interesting comparison of different hash functions families:
https://en.wikipedia.org/wiki/SHA-3#Comparison_of_SHA_functions

"The algorithm takes as input a message of arbitrary length and produces
as output a 128-bit "fingerprint" or "message digest" of the input.
It is conjectured that it is computationally infeasible to produce
two messages having the same message digest, or to produce any
message having a given prespecified target message digest. The MD5
algorithm is intended for digital signature applications, where a
large file must be "compressed" in a secure manner before being
encrypted with a private (secret) key under a public-key cryptosystem
such as RSA."

In [1]:
from bitarray import bitarray
import struct

def add_32(a,b):
    return (a+b)% (2**32)

#python bitwise operations
#https://www.journaldev.com/26737/python-bitwise-operators
#&AND; |OR; ^XOR;~Ones’ Compliment; <<LShift; >>RShift

#Let X <<< s denote the 32-bit value obtained by circularly shifting (rotating) X left by s bit positions.
#shift left n OR shift right (32-n)
def rotate_left(x,n):
    return (x<<n)| (x>>(32-n))

## Definitions of four auxiliary functions for step_4
Each take as input three 32-bit words and produce as output one 32-bit word.

* F(X,Y,Z) = XY v not(X) Z
* G(X,Y,Z) = XZ v Y not(Z)
* H(X,Y,Z) = X xor Y xor Z
* I(X,Y,Z) = Y xor (X v not(Z))

In [2]:
def F(x,y,z):
    return (x& y)| (~x& z)
def G(x,y,z):
    return (x& z)| (y& ~z)
def H(x,y,z):
    return x^y^z
def I(x,y,z):
    return y^ (x| ~z)

## Definitions of tables for step_4

**shift_amounts** specifies the per-round shift amounts

**sine_table[i]** denote the i-th element of the table, which is equal to the integer part of 

(2\*\*32)\*abs(sin(i)), where i is in radians.

In [3]:
SHIFT_AMOUNTS={
    'round1': [7, 12, 17, 22],
    'round2': [5,  9, 14, 20],
    'round3': [4, 11, 16, 23],
    'round4': [6, 10, 15, 21]
}

In [4]:
import math
SINE_TABLE=[math.floor((2**32)* abs(math.sin(i + 1))) for i in range(64)]

## Definitions of rounds for step_4

In [5]:
round_index={
    range(0 ,16):'round1',
    range(16,32):'round2',
    range(32,48):'round3',
    range(48,64):'round4'
}
def get_round_index(i):
    for k,v in round_index.items():
        if i in k:
            return v

In [6]:
k1=lambda i: i
k2=lambda i: ((5*i)+1)% 16
k3=lambda i: ((3*i)+5)% 16
k4=lambda i: (7*i)% 16

round_funcs={
    'round1':(k1,F),
    'round2':(k2,G),
    'round3':(k3,H),
    'round4':(k4,I)
}

## 3.1 Step 1. Append Padding Bits

   The message is "padded" (extended) so that its length (in bits) is
   congruent to 448, modulo 512. That is, the message is extended so
   that it is just 64 bits shy of being a multiple of 512 bits long.
   Padding is always performed, even if the length of the message is
   already congruent to 448, modulo 512.

   Padding is performed as follows: a single "1" bit is appended to the
   message, and then "0" bits are appended so that the length in bits of
   the padded message becomes congruent to 448, modulo 512. In all, at
   least one bit and at most 512 bits are appended.
   
##   3.2 Step 2. Append Length

   A 64-bit representation of b (the length of the message before the
   padding bits were added) is appended to the result of the previous
   step. In the unlikely event that b is greater than 2^64, then only
   the low-order 64 bits of b are used. (These bits are appended as two
   32-bit words and appended low-order word first in accordance with the
   previous conventions.)

   At this point the resulting message (after padding with bits and with
   b) has a length that is an exact multiple of 512 bits. Equivalently,
   this message has a length that is an exact multiple of 16 (32-bit)
   words. Let M[0 ... N-1] denote the words of the resulting message,
   where N is a multiple of 16.

In [7]:
def step_1_padding(m_str):
    m_array = bitarray(endian="big")
    m_array.frombytes(m_str.encode("utf-8"))

    m_array.append(1)
    while len(m_array)% 512 != 448:
        m_array.append(0)
    
    return bitarray(m_array, endian="little")

def step_2_concat_len(m_str,m_array):
    length = (8*len(m_str))% (2**64)
    
    #https://stackoverflow.com/questions/42652436/how-to-convert-ctypes-to-bytes
    #https://docs.python.org/3.7/library/struct.html
    #Q: unsigned long long; < little; > big
    appendix = bitarray(endian="little")
    appendix.frombytes(struct.pack("<Q", length))

    result = m_array.copy()
    result.extend(appendix)
    return result

In [8]:
#a=01100001 (utf-8 binary)
len(step_1_padding('a')),step_1_padding('a')

(448,
 bitarray('1000011000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'))

In [9]:
m_str='a'
m_array=step_1_padding(m_str)
len(step_2_concat_len(m_str,m_array)),step_2_concat_len(m_str,m_array)

(512,
 bitarray('10000110000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000'))

## 3.3 Step 3. Initialize MD Buffer

   A four-word buffer (A,B,C,D) is used to compute the message digest.
   Here each of A, B, C, D is a 32-bit register. These registers are
   initialized to the following values in hexadecimal, low-order bytes
   first):

In [10]:
def step_3_init_state():
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476
    #here simple dict with keys 'A','B','C','D' and following values
    return {'A':A,'B':B,'C':C,'D':D}

## 3.4 Step 4. Process Message in 16-Word Blocks

In [11]:
def step_4_calc(state,m_padded):
    #N is number of words
    N = len(m_padded)// 32

    #Chunks of 512 bits (32*16)
    for ch_i in range(N// 16):
        #ch_i*512: offset chunk; i offset word; 32 word length; 16 block length
        X = [m_padded[((ch_i*512)+(i*32)):((ch_i*512)+(i*32)+32)] for i in range(16)]

        #to int
        X = [int.from_bytes(word.tobytes(), byteorder="little") for word in X]

        A = state['A']
        B = state['B']
        C = state['C']
        D = state['D']

        #4 rounds x 16 operations
        for i in range(4*16):
            round_index=get_round_index(i)
            ki,FGHI=round_funcs.get(round_index)
            
            #shift array
            s=SHIFT_AMOUNTS[round_index]
            #value dependent on i
            s_val=s[i % 4]
            
            #ki returns mod 16
            k=ki(i)
            #tmp value for one of F,G,H or I func
            tmp=FGHI(B,C,D)

            #tmp := tmp + A + SINE_TABLE[i] + Message[k]  // M[k] must be a 32-bits block
            #consecutive addition
            tmp = add_32(tmp, X[k])
            tmp = add_32(tmp, SINE_TABLE[i])
            tmp = add_32(tmp, A)
            
            #add B and rotate s_val
            tmp = rotate_left(tmp, s_val)
            tmp = add_32(tmp, B)

            # rotate the registers one step
            A = D
            D = C
            C = B
            B = tmp

        #update state
        state['A'] = add_32(state['A'], A)
        state['B'] = add_32(state['B'], B)
        state['C'] = add_32(state['C'], C)
        state['D'] = add_32(state['D'], D)
        
    return state

## 3.5 Step 5. Output

   The message digest produced as output is A, B, C, D. That is, we
   begin with the low-order byte of A, and end with the high-order byte
   of D.

   This completes the description of MD5. A reference implementation in
   C is given in the appendix.

In [12]:
def format_state_val(val):
    F=struct.unpack("<I",struct.pack(">I",val))[0]
    return format(F, '08x')
def step_5_print(out_state):
    hex_strings=[format_state_val(out_state[k]) for k in ['A','B','C','D']]
    return "".join(hex_strings)

## MD5 hash function:

In [13]:
def md5_hash(m_str):
    m_array=step_1_padding(m_str)
    m_padded=step_2_concat_len(m_str,m_array)
    internal_state=step_3_init_state()
    out_state=step_4_calc(internal_state,m_padded)
    return step_5_print(out_state)

## MD5 test suite from paper:
* MD5 ("") = d41d8cd98f00b204e9800998ecf8427e
* MD5 ("a") = 0cc175b9c0f1b6a831c399e269772661
* MD5 ("abc") = 900150983cd24fb0d6963f7d28e17f72

In [14]:
test_suite=[
    ('d41d8cd98f00b204e9800998ecf8427e',""),
    ('0cc175b9c0f1b6a831c399e269772661',"a"),
    ('900150983cd24fb0d6963f7d28e17f72', "abc"),
    ('f96b697d7cb7938d525a2f31aaf161d0', "message digest"),
    ('c3fcd3d76192e4007dfb496cca67e13b', "abcdefghijklmnopqrstuvwxyz"),
    ('d174ab98d277d9f5a5611c2c9f419d9f', 
     "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"),
    ('57edf4a22be3c955ac49da2e2107b67a',
    "12345678901234567890123456789012345678901234567890123456789012345678901234567890")
]

In [15]:
for h,m_str in test_suite:
    assert md5_hash(m_str) == h

In [16]:
import hashlib
#implementations in other languages https://rosettacode.org/wiki/MD5/Implementation
print(hashlib.md5('test\n'.encode()).hexdigest())
print(md5_hash('test\n'))

d8e8fca2dc0f896fd7cb4cb0031ba249
d8e8fca2dc0f896fd7cb4cb0031ba249
